In [11]:
import numpy as np 
import pandas as pd 
import openpyxl

In [12]:
df = pd.read_excel(r"C:\Desktop\predictiveAnalysis\data\raw\who_air_quality.xlsx", engine = "openpyxl")

In [13]:
df.head(5)

,who_region,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
0,4_Eur,ESP,Spain,A Coruna/ESP,2013.0,"V4.0 (2018), V4.0 (2018), V4.0 (2018), V4.0 (2...",23.238,11.491,28.841,87.0,46.0,93.0,"Urban, Urban, Suburban",NaN,NaN,246146.0,"manual, manual, manual, manual",43.367900,-8.418571,1
1,4_Eur,ESP,Spain,A Coruna/ESP,2014.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023)",27.476,15.878,19.575,96.0,88.0,95.0,"Urban, Urban, Suburban",NaN,NaN,247604.0,NaN,43.368033,-8.418233,1
2,4_Eur,ESP,Spain,A Coruna/ESP,2015.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",25.515,14.004,22.731,98.0,71.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
3,4_Eur,ESP,Spain,A Coruna/ESP,2016.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",23.057,13.160,20.204,98.0,98.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
4,4_Eur,ESP,Spain,A Coruna/ESP,2017.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",26.849,14.114,21.543,97.0,97.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1


## Cleaning of data

In [14]:
df = df.drop(columns = ['version','reference', 'web_link', 'population_source'])

In [ ]:
# spliiting the city 
df['city'] = df['city'].str.split('/').str[0]


In [ ]:
# the year is in float so converting to int 
df['year'] = df['year'].astype('Int64')


In [ ]:
# type_of_stations contains multiple values like citynae, country which is related to other col.. so splitting it 
df['type_of_stations'] = df['type_of_stations'].str.split(',').str[0].str.strip()


In [18]:
df['who_region'] = df['who_region'].str.split('_').str[1]


In [19]:
df.head()

,who_region,iso3,country_name,city,year,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,population,latitude,longitude,who_ms
0,Eur,ESP,Spain,A Coruna,2013,23.238,11.491,28.841,87.0,46.0,93.0,Urban,246146.0,43.367900,-8.418571,1
1,Eur,ESP,Spain,A Coruna,2014,27.476,15.878,19.575,96.0,88.0,95.0,Urban,247604.0,43.368033,-8.418233,1
2,Eur,ESP,Spain,A Coruna,2015,25.515,14.004,22.731,98.0,71.0,98.0,Urban,247604.0,43.370375,-8.422900,1
3,Eur,ESP,Spain,A Coruna,2016,23.057,13.160,20.204,98.0,98.0,98.0,Urban,247604.0,43.370375,-8.422900,1
4,Eur,ESP,Spain,A Coruna,2017,26.849,14.114,21.543,97.0,97.0,98.0,Urban,247604.0,43.370375,-8.422900,1


## handling missing values

In [21]:
df.isnull().sum()

who_region                0
iso3                      0
country_name              0
city                      0
year                      3
pm10_concentration    11426
pm25_concentration    18368
no2_concentration     13164
pm10_tempcov          17695
pm25_tempcov          23508
no2_tempcov           16696
type_of_stations      16767
population            17665
latitude                  0
longitude                 0
who_ms                    0
dtype: int64

In [22]:
df['year'] = df['year'].fillna(df['year'].bfill())

In [40]:
df['type_of_stations'] = df['type_of_stations'].bfill().ffill()


In [41]:
# skewness for checking asymmetry in data 

df[['pm10_concentration','pm25_concentration','no2_concentration','population']].skew()


pm10_concentration      5.256437
pm25_concentration      4.840755
no2_concentration     129.582263
population              9.900745
dtype: float64

In [42]:
pollution_cols = [
    'pm10_concentration', 'pm25_concentration', 'no2_concentration'
]
for col in pollution_cols:
    df[col] = df[col].fillna(df[col].median())

In [43]:
df['population'] = df['population'].fillna(df['population'].median())

In [44]:
df[['pm10_tempcov', 'pm25_tempcov', 'no2_tempcov']].skew()

pm10_tempcov   -4.191174
pm25_tempcov   -3.931801
no2_tempcov    -4.167733
dtype: float64

In [45]:
tempcov_cols = [
    'pm10_tempcov', 'pm25_tempcov', 'no2_tempcov'
]
for col in tempcov_cols:
    df[col] = df[col].fillna(df[col].median())

## Outlier detection

In [48]:
num_cols = ['pm10_concentration','pm25_concentration','no2_concentration','pm10_tempcov',
                'pm25_tempcov','no2_tempcov','population']

In [50]:
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

In [52]:
df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

## Feature Engineering

In [73]:
df['pollution_index'] = (df['pm10_concentration'] + 
                         df['pm25_concentration'] + 
                         df['no2_concentration']) / 3


In [68]:
df['pollution_per_person'] = (df['pm10_concentration']/df['population'] + 
                         df['pm25_concentration']/df['population'] + 
                         df['no2_concentration']) / df['population']


In [74]:
df = df.sort_values(by=['city', 'year'])

df['pollution_index_yearly_change'] = df.groupby('city')['pollution_index'].diff()
df['pollution_index_yearly_change'] = df['pollution_index_yearly_change'].fillna(0)

## Scaling

In [ ]:


from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

cols = ['pm10_concentration', 'pm25_concentration', 'no2_concentration',
        'pm10_tempcov', 'pm25_tempcov', 'no2_tempcov', 'population', 'pollution_per_person', 'pollution_index']

scaler = StandardScaler()
df[cols] = scaler.fit_transform(df[cols])


## Encoding

In [ ]:


ohe = OneHotEncoder(sparse_output=False)
le = LabelEncoder()

df['type_of_stations'] = le.fit_transform(df['type_of_stations'])

df['who_region'] = ohe.fit_transform(df[['who_region']])
df['country_name'] = ohe.fit_transform(df[['country_name']])
df['iso3'] = ohe.fit_transform(df[['iso3']])
df['city'] = ohe.fit_transform(df[['city']])


In [92]:
df.head()

,who_region,iso3,country_name,city,year,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,population,latitude,longitude,who_ms,pollution_per_person,pollution_index,pollution_index_yearly_change
0,1.0,1.0,1.0,0.0,2013,-0.170206,-0.354191,0.504059,-0.317035,-2.475293,0.089188,9,-0.750809,43.367900,-8.418571,1,-0.040130,0.041835,0.000000
1,1.0,1.0,1.0,0.0,2014,-0.002639,-0.023508,0.067105,0.260630,-0.169461,0.199629,9,-0.747451,43.368033,-8.418233,1,-0.041330,0.025924,-0.213667
2,1.0,1.0,1.0,0.0,2015,-0.080175,-0.164766,0.215932,0.389000,-1.102774,0.365289,9,-0.747451,43.370375,-8.422900,1,-0.040929,0.009070,-0.226333
3,1.0,1.0,1.0,0.0,2016,-0.177362,-0.228385,0.096767,0.389000,0.379546,0.365289,9,-0.747451,43.370375,-8.422900,1,-0.041250,-0.135616,-1.943000
4,1.0,1.0,1.0,0.0,2017,-0.027430,-0.156475,0.159910,0.324815,0.324645,0.365289,9,-0.747451,43.370375,-8.422900,1,-0.041080,0.015425,2.028333


In [94]:
import os

output_dir = 'data/processed'
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, 'processed_data.csv')
df.to_csv(output_file, index=False)